In [0]:
from pyspark.sql.functions import col, rand

In [0]:
df1 = spark.range(1, 1000000, 1, 10).select(col("id"), rand(10).alias("Attribute_1"))
df1.write.format("parquet").saveAsTable("non_bucketed1")
df1.write.format("parquet").bucketBy(10, "id").saveAsTable("bucketed1")

In [0]:
df2 = spark.range(1, 2000000, 2, 10).select(col("id"), rand(10).alias("Attribute_2"))
df2.write.format("parquet").saveAsTable("non_bucketed2")
df2.write.format("parquet").bucketBy(10, "id").saveAsTable("bucketed2")

####Turn off broadcast hash join

In [0]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

#### Join of two non bucketed tables

In [0]:
spark.table("non_bucketed1").join(spark.table("non_bucketed2"), "id").show()

+-----+-------------------+--------------------+
|   id|        Attribute_1|         Attribute_2|
+-----+-------------------+--------------------+
|   29|0.19412650105821194| 0.47649428738170896|
| 1677| 0.9708446180801303| 0.43885423251777256|
| 1697|0.24049784604366053|  0.8419374040517177|
| 2453| 0.7256288093043027|  0.1946845167986896|
| 2509|0.01107612292305682|  0.1818109453586787|
| 2529| 0.4180559794927762| 0.09421624963407049|
| 2927| 0.4359564206495279| 0.37185344032768264|
| 3091| 0.5106400071204449|  0.7524722512897478|
| 4823|0.26778225072447837|  0.5645895496096658|
| 5385|0.41606059099070203| 0.18938931242519907|
| 5409| 0.3601627029946787|0.042999761708867656|
| 6721| 0.4791487959708005|  0.1261285677074625|
| 7225|  0.975552759622486|   0.466678271484284|
| 7279| 0.7589055862340827|   0.791002637186319|
| 7747| 0.6127045498954239|  0.9976503326060169|
| 8075| 0.3449904905435699|  0.9513315788999599|
| 9233| 0.2592681841652619| 0.09607149370033552|
| 9715|0.03590715437

#### Join of a bucketed table with non-bucketed table

In [0]:
spark.table("bucketed2").join(spark.table("bucketed2"), "id").show()

+---+-------------------+-------------------+
| id|        Attribute_2|        Attribute_2|
+---+-------------------+-------------------+
| 27| 0.8133304803837667| 0.8133304803837667|
| 29|0.47649428738170896|0.47649428738170896|
| 61| 0.4309638186000935| 0.4309638186000935|
| 63| 0.6606700902936103| 0.6606700902936103|
| 83| 0.5030522433081416| 0.5030522433081416|
|115|0.08969927916746101|0.08969927916746101|
|125| 0.5771966653256273| 0.5771966653256273|
|153| 0.2670434277204943| 0.2670434277204943|
|163|0.05984926325784168|0.05984926325784168|
|171| 0.8875227420980584| 0.8875227420980584|
|229| 0.9087820885752338| 0.9087820885752338|
|247| 0.7702378879843016| 0.7702378879843016|
|303| 0.5632218469790521| 0.5632218469790521|
|389| 0.9001144377699173| 0.9001144377699173|
|391|0.21913808726684847|0.21913808726684847|
|397|0.09250526938750026|0.09250526938750026|
|425|  0.641461246802741|  0.641461246802741|
|427| 0.9603113546372618| 0.9603113546372618|
|445|0.12846070050922387|0.1284607

#### Join of two bucketed tables with same number of buckets on same column

In [0]:
spark.table("bucketed2").join(spark.table("non_bucketed1"), "id").show()

+---+-------------------+--------------------+
| id|        Attribute_2|         Attribute_1|
+---+-------------------+--------------------+
| 27| 0.8133304803837667| 0.06622344831941485|
| 29|0.47649428738170896| 0.19412650105821194|
| 61| 0.4309638186000935|  0.5352770539384141|
| 63| 0.6606700902936103|  0.5771966653256273|
| 83| 0.5030522433081416| 0.47415236401346694|
|115|0.08969927916746101|  0.9087820885752338|
|125| 0.5771966653256273|  0.7086532984353837|
|153| 0.2670434277204943| 0.20399627293507627|
|163|0.05984926325784168|  0.9832199149707848|
|171| 0.8875227420980584| 0.10630723267408737|
|229| 0.9087820885752338|  0.1812611897448443|
|247| 0.7702378879843016|  0.8455718587337636|
|303| 0.5632218469790521| 0.18807110811314076|
|389| 0.9001144377699173|0.013610376252277123|
|391|0.21913808726684847|  0.7099177814100067|
|397|0.09250526938750026|  0.7163561883978399|
|425|  0.641461246802741|  0.9006428845075235|
|427| 0.9603113546372618| 0.44672850066745406|
|445|0.128460